# Dropout

神经网络中如果产生过拟合，说明每一层过分依赖于其前一层中特征的确切配置，为了防止神经网络过度依赖于之前层的特征的激活路径，我们可以以$0.5\%$的概率随机丢弃掉某一隐层中的部分节点，因此给定一个具有$n$个节点的网络，我们大致相当于从一个部分节点被丢弃的$2^n$种网络中随机均匀采样。

<img src="http://www.angzz.top:8888/notebooks/Open-Source-Project/mxnet-the-straight-dope/img/dropout.png" width="300">

$drpoout$起作用的直观解释就是：因为每次都要随机选择丢弃的节点，因此每一层中的表示都不能依赖于前一层中节点的确切值。

随机失活在约束网络复杂度同时，还是一种针对深度模型的高效集成学习(ensemble learning)的方法。传统神经网络中，由于神经元间的互联，对于某单个神经元来说，其反向传导来的梯度信息同时也受到其他神经元的影响，可谓“牵一发而动全身”。这就是所谓的“复杂协同适应”效应。随机失活的提出正 是一定程度上缓解了神经元之间复杂的协同适应，降低了神经元间依赖，避免了网络过拟合的发生。其原理非常简单:**对于某层的每个神经元，在训练阶段 均以概率$p$随机将该神经元权重置(故被称作“随机失活”)，测试阶段所有神经元均呈激活态，但其权重需乘$(1-p)$以保证训练和测试阶段各自权重拥有相同的期望。**

[**浅析Dropout层中为什么在训练阶段dropout后在测试阶段要进行rescale**](https://www.zhihu.com/question/61751133)

**<font color="red">分析如下</font>**：当模型使用了dropout layer以后，我们以概率$p$丢弃某一层中的部分神经元，那么训练时候只有占比为$1-p$的隐藏层单元参与训练。而在预测的时候，如果我们需要所有的隐藏层单元都要参与进来，那么其得到的结果相比训练时平均要大$\frac{1}{1-p}$，为了避免这种情况，就需要测试的时候将输出结果乘以$1-p$使得下一层的输入规模(**即期望**)保持不变。

而利用反向随机失活(inverted dropout)，我们可以在训练的时候直接将dropout后留下的权重扩大$\frac{1}{1-p}$倍，这样就可以使得结果的scale保持不变，而在预测的时候就不需要做额外的操作了。这样更方便一些。

**<font color="red">另一种理解方式</font>**：假设我们设置dropout probability为p，那么该层大约有比例为p的单元会被丢弃，因为每个神经元是否被丢弃就是一次伯努利实验，因此若该层有n个神经元，则该层的dropout概率相当于n重伯努利实验，其服从伯努利分布，而伯努利分布的期望就是np。

又因为有$z^l = w^la^{l-1} + b^l$，那么当$l-1$层有比例为p的单元被丢弃后，$a^{l-1}$的规模会变为原来的1-p倍，因此为了保证$l$层的输出$z^l$的期望不变，所以要在$a^{l-1}$与dropout矩阵乘积后，要除以$1-p$。

**<font color="red">第三种理解方式</font>**：10个人拉一个10吨车，第一次（训练时），只有5个人出力（有p=0.5的人被dropout了），那么这5个人每个人出力拉2吨。第二次（预测时），10个人都被要求出力，这次每个人出的力就是2*（1-0.5）=1吨了。

In [1]:
import mxnet as mx
import numpy as np

from mxnet import nd
from mxnet import gluon
from mxnet import autograd
mx.random.seed(1)

%matplotlib inline 
import matplotlib.pyplot as plt
import utils

In [2]:
ctx = mx.cpu()

In [3]:
batch_size = 128
train_data, test_data = utils.load_dataset(batch_size, data_type='fashion_mnist')

In [4]:
num_inputs = 784
num_hidden = 256
num_outputs = 10
num_examples = 60000
weight_scale = 0.01

W1 = nd.random.normal(shape=(num_inputs, num_hidden), scale=weight_scale)
b1 = nd.random.normal(shape=(num_hidden, ), scale=weight_scale)

W2 = nd.random.normal(shape=(num_hidden, num_hidden), scale=weight_scale)
b2 = nd.random.normal(shape=(num_hidden, ), scale=weight_scale)

W3 = nd.random.normal(shape=(num_hidden, num_outputs), scale=weight_scale)
b3 = nd.random.normal(shape=(num_outputs, ), scale=weight_scale)

params = [W1, b1, W2, b2, W3, b3]

for param in params:
    param.attach_grad()

In [5]:
def relu(X):
    return nd.maximum(X, nd.zeros_like(X))

## Dropout

In [6]:
def dropout(X, drop_prob = 0.0):
    keep_prob = 1 - drop_prob
    mask = nd.random.uniform(0.0, 1.0, X.shape) > drop_prob
    if keep_prob == 0.0:
        scale = 1
    else:
         # 保证 E[dropout(X)] == X
        scale = 1 / (1 - drop_prob)
    return X * mask * scale

In [7]:
a = nd.arange(20).reshape((5, 4)) + 1
dropout(a, 0.0)


[[  1.   2.   3.   4.]
 [  5.   6.   7.   8.]
 [  9.  10.  11.  12.]
 [ 13.  14.  15.  16.]
 [ 17.  18.  19.  20.]]
<NDArray 5x4 @cpu(0)>

In [8]:
dropout(a, 0.5)


[[  0.   4.   6.   0.]
 [  0.   0.  14.  16.]
 [ 18.   0.   0.   0.]
 [ 26.   0.  30.  32.]
 [  0.   0.   0.  40.]]
<NDArray 5x4 @cpu(0)>

In [9]:
dropout(a, 1.0)


[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
<NDArray 5x4 @cpu(0)>

## Softmax_cross_entropy (维持数值稳定性)

In [10]:
def softmax(ylinear):
    yexp = nd.exp(ylinear - nd.max(ylinear))
    partition = yexp / nd.nansum(yexp, axis=1, exclude=True)
    return partition

def softmax_cross_entropy(yhat, y):
    return -nd.nansum(y * nd.log(softmax(yhat)), axis=1, exclude=True)

## 定义模型

In [11]:
def net(X):
    X = X.reshape((-1, 784))
    h1 = nd.dot(X, W1) + b1
    h1 = relu(h1)
    h1 = dropout(h1, 0.2)
    
    h2 = nd.dot(h1, W2) + b2
    h2 = relu(h2)
    h2 = dropout(h2, 0.5)
    
    h3 = nd.dot(h2, W3) + b3
    return h3

In [ ]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

## 训练

In [ ]:
epochs = 10
learning_rate = 0.001

for epoch in range(epochs):
    cumulative_loss = .0
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        label_one_hot = nd.one_hot(label, 10)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label_one_hot)
        loss.backward()
        SGD(params, learning_rate)
        cumulative_loss += nd.sum(loss).asscalar()
        
    train_acc = utils.evaluate_accuracy_scratch(train_data, net, ctx)
    test_acc = utils.evaluate_accuracy_scratch(test_data, net, ctx)
    print("Epoch %s, Train loss %s, Train acc %s, Test acc %s." 
          % (epoch, cumulative_loss / num_examples, train_acc, test_acc))

Epoch 0, Train loss 3.82514266561, Train acc 0.64995, Test acc 0.6511.
Epoch 1, Train loss 3.22903592224, Train acc 0.683583, Test acc 0.6842.
Epoch 2, Train loss 3.11905423635, Train acc 0.764083, Test acc 0.7641.
Epoch 3, Train loss 3.06101858927, Train acc 0.80915, Test acc 0.8086.
Epoch 4, Train loss 3.02602425537, Train acc 0.839517, Test acc 0.8334.
Epoch 5, Train loss 2.99819885406, Train acc 0.8583, Test acc 0.8477.
Epoch 6, Train loss 2.97827471339, Train acc 0.85755, Test acc 0.848.
Epoch 7, Train loss 2.96405434977, Train acc 0.862467, Test acc 0.854.
Epoch 8, Train loss 2.9488596817, Train acc 0.872, Test acc 0.8635.
